<a href="https://colab.research.google.com/github/lupries/TAGN2/blob/master/scripts/DataloaderDAVIS16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/davisvideochallenge/davis.git

Cloning into 'davis'...
remote: Enumerating objects: 1003, done.
remote: Total 1003 (delta 0), reused 0 (delta 0), pack-reused 1003
Receiving objects: 100% (1003/1003), 1.66 MiB | 5.01 MiB/s, done.
Resolving deltas: 100% (315/315), done.


In [2]:
! davis/data/get_davis.sh

--2019-11-22 10:01:11--  https://graphics.ethz.ch/Downloads/Data/Davis/DAVIS-data.zip
Resolving graphics.ethz.ch (graphics.ethz.ch)... 129.132.145.103
Connecting to graphics.ethz.ch (graphics.ethz.ch)|129.132.145.103|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cgl.ethz.ch/Downloads/Data/Davis/DAVIS-data.zip [following]
--2019-11-22 10:01:12--  https://cgl.ethz.ch/Downloads/Data/Davis/DAVIS-data.zip
Resolving cgl.ethz.ch (cgl.ethz.ch)... 129.132.145.103
Connecting to cgl.ethz.ch (cgl.ethz.ch)|129.132.145.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1958016855 (1.8G) [application/zip]
Saving to: ‘DAVIS-data.zip’

DAVIS-data.zip      100%[===================>]   1.82G  19.7MB/s    in 99s     

2019-11-22 10:02:52 (18.8 MB/s) - ‘DAVIS-data.zip’ saved [1958016855/1958016855]

0cb3cf9c5617209fa3cc4794e52a2ffa
Checksum is correct.
Unzipping...
Archive:  DAVIS-data.zip
   creating: DAVIS/
   creating: DAVIS

In [37]:
! cd davis && ./configure.sh && make -C build/release


------------------------------------------------------------------------
Generating Release Makefile
------------------------------------------------------------------------
-- Boost version: 1.65.1
-- Found the following Boost libraries:
--   python
-- Found PythonInterp: /usr/local/bin/python (found suitable version "2.7.15", minimum required is "2.7.6") 
-- Found PythonInterp: /usr/local/bin/python (found version "2.7.15") 
-- Boost version: 1.65.1
-- Found the following Boost libraries:
--   python
-- Found Boost
-- Configuring done
-- Generating done
-- Build files have been written to: /content/davis/build/release

------------------------------------------------------------------------
Generating Debug Makefile
------------------------------------------------------------------------
-- Boost version: 1.65.1
-- Found the following Boost libraries:
--   python
-- Found PythonInterp: /usr/local/bin/python (found suitable version "2.7.15", minimum required is "2.7.6") 
-- Found Pyt

In [1]:
import sys
sys.path.append('/content/davis/python/lib')
sys.path.append('/content/davis/build/release')
sys.path.remove('')
sys.path

['/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/davis/python/lib',
 '/content/davis/build/release']

In [0]:
from davis.dataset import *
from davis.dataset.utils import *

In [9]:
seg = db_read_info().sequences[0]
dataloader = DAVISSegmentationLoader(cfg, 'bear')
dataloader[0]

TypeError: ignored

In [0]:
%pycat /content/davis/python/lib/davis/config.py

In [46]:
%%writefile /content/davis/python/lib/davis/dataset/loader.py
#!/usr/bin/env python
# -------------------------------------------------------------------------------
# A Benchmark Dataset and Evaluation Methodology for Video Object Segmentation
# Copyright (c) 2016 Federico Perazzi
# Licensed under the BSD License [see LICENSE for details]
# Written by Federico Perazzi
# ------------------------------------------------------------------------------

__author__ = 'federico perazzi'
__version__ = '1.0.0'

########################################################################
#
# Interface for accessing the DAVIS dataset.
#
# DAVIS is a video dataset designed for segmentation. The API implemented in
# this file provides functionalities for loading, parsing and visualizing
# images and annotations available in DAVIS. Please visit
# [https://graphics.ethz.ch/~perazzif/davis] for more information on DAVIS,
# including data, paper and supplementary material.
#
# The following API functions are defined:
#       DAVISSegmentationLoader - Class that loads DAVIS data.
#               images           - return input images.
#               masks                    - return segmentation masks.
#               iternames  - return iterator over tuples of image and mask filenames.
#               iteritems  - return iterator over tuples of images and masks.
#
#       DAVISAnnotationLoader - Class that loads DAVIS annotations and perform evaluation.
#               eval                    - perform evaluation of region similarity (J),
#                                                               boundary accuracy (F) and temporal stability (T)
########################################################################

import os
import copy
import skimage.io
import numpy as np

from davis import log
#from davis.measures import db_eval_boundary,db_eval_iou,db_eval_t_stab

def _load_annotation(fname,img_num=0):
        return skimage.io.imread(fname,as_grey=True)

class DAVISSegmentationLoader(object):
        """ Helper class for accessing the DAVIS dataset.

        Arguments:
                cfg: configuration file provided in davis.config
                sequence (string): sequence name
                masks_dir(string): path to segmentation images.
                ext_im   (string): images file extension
                ext_an   (string): annotations file extension
                load_func        : function to load annotations

        Functions:
                eval       : evaluate sequence
                images           : return input images
          masks                  : return segmentation masks
          iternames  : return iterator over tuples of image and mask filenames
                iteritems  : return iterator over tuples of images and masks

        """
        def __init__(self,cfg,sequence,masks_dir=None,ext_im="jpg",
                        ext_an="png", load_func=_load_annotation):

                super(DAVISSegmentationLoader, self).__init__()

                self._cfg                               = cfg
                self.name                               = sequence

                self._ext_an             = ext_an
                self._ext_im             = ext_im
                self._load_func  = load_func

                self.images_dir = os.path.join(
                                self._cfg.PATH.SEQUENCES_DIR,self.name)

                if masks_dir is None:
                        self.masks_dir = os.path.join(self._cfg.PATH.ANNOTATION_DIR,self.name)
                else:
                        self.masks_dir = os.path.join(masks_dir,self.name)

                assert os.path.isdir(self.images_dir),\
                                "Couldn't find folder: %s"%self.images_dir

                #########################################
                # LOAD IMAGES AND MASKS
                #########################################
                self._images = skimage.io.ImageCollection(self.images_dir+"/*%s"%self._ext_im)

                self._masks = skimage.io.ImageCollection(self.masks_dir+"/*%s"%self._ext_an,
                                load_func=_load_annotation)

                #assert len(self._masks) != 0 and len(self._images) != 0
                masks_frames = map(lambda fn:
                                int(os.path.splitext(os.path.basename(fn))[0]),self._masks.files)

                # Loading the ground-truth annotations
                if masks_dir is None:
                        assert len(self._images) == len(self._masks)
                else:
                        assert masks_frames[0] == 0 or masks_frames[0] == 1 and \
                                        len(masks_frames) == masks_frames[-1] - masks_frames[0] + 1

                        self._images = self._images[masks_frames[0]:masks_frames[-1]+1]

                self._frames = list(range(masks_frames[0],
                        len(self._images)+masks_frames[0]))

                assert len(self._frames) == len(self._masks) == len(self._images)

                # Compute bounding boxes
                self._bbs = []
                for mask in self._masks:
                        coords = np.where(mask!=0)
                        if len(coords[0]) <=1:
                                self._bbs.append(None)
                        else:
                                tl = np.min(coords[1]),np.min(coords[0])
                                br = np.max(coords[1]),np.max(coords[0])

                                self._bbs.append((tl[0],tl[1],br[0],br[1]))

                # FINAL SANITY CHECK
                image_frames = map(lambda fn:
                                int(os.path.splitext(os.path.basename(fn))[0]),self._images.files)

                assert np.allclose(image_frames,masks_frames)


        def __getitem__(self, n):
                """
                Return selected image(s) in the collection.
                Loading is done on demand.
                """

                if hasattr(n, '__index__'):
                        n = n.__index__()

                if type(n) not in [int, slice]:
                 raise TypeError('slicing must be with an int or slice object')

                if type(n) is int:
                        n = slice(n,n+1) # Cast to slice

                fidx = range(len(self))[n]

                # A slice object was provided.
                new_ic                           = copy.copy(self)
                new_ic._masks  = self._masks[n]
                new_ic._images = self._images[n]

                new_ic._frames = [self._frames[i] for i in fidx]

                return new_ic

        def __len__(self):
                return len(self._images)

        def __str__(self):
                return self.name

        @property
        def masks(self):
                return self._masks

        @property
        def images(self):
                return self._images

        #######################################
        # ITERATORS
        #######################################
        def iternames(self):
                for im,ma in zip(self.images.files,self._masks.files):
                        yield im,ma

        def iteritems(self):
                for im,ma in zip(self._images,self._masks):
                        yield im,ma

class DAVISAnnotationLoader(DAVISSegmentationLoader):

        """ Helper class for accessing the DAVIS dataset.

        Arguments:
                cfg: configuration file provided in davis.config
                sequence (string): sequence name
                ext_im   (string): images file extension
                ext_an   (string): annotations file extension
                load_func        : function to load annotations

        Functions: (see DAVISSegmentationLoader documentation)
                eval: evaluate sequence

        """

        def __init__(self,cfg,sequence,ext_im="jpg",
                        ext_an="png",load_func=_load_annotation):

                super(DAVISAnnotationLoader, self).__init__(
                                cfg,sequence,None,ext_im,ext_an,load_func)

        def _eval(self,db_segmentation,eval_func,measure,scale=1):
                annotations = self._masks[1:-1]

                # Strip of first and last frame if available
                segmentation = db_segmentation._masks[
                                1-db_segmentation._frames[0]:len(annotations)+1-db_segmentation._frames[0]]

                assert len(annotations) == len(segmentation)

                if measure == 'T':
                        magic_number = 5.0
                        X = np.array([np.nan]+[eval_func(an,sg)*magic_number for an,sg
                                in zip(segmentation[:-1],segmentation[1:])] + [np.nan])
                else:
                        X = np.array([np.nan]+[eval_func(an,sg) for an,sg
                                        in zip(annotations,segmentation)] + [np.nan])

                from utils import db_statistics
                M,O,D = db_statistics(X)

                if measure == 'T': O = D = np.nan

                return X,M,O,D
'''
        def eval(self,db_segmentation,measure='J'):

                """ Evaluate sequence.

                Arguments:
                        db_segmentation (DAVISSegmentationLoader) : sequence file to be evaluated.
                        measure         (string: 'J','F','T')     : measure to be computed

                Returns:
                        X: per-frame measure evaluation.
                        M: mean   of per-frame measure.
                        O: recall of per-frame measure.
                        D: decay  of per-frame measure.

                """

                if measure == 'J':
                        return self._eval(db_segmentation,db_eval_iou,measure)
                elif measure=='F':
                        return self._eval(db_segmentation,db_eval_boundary,measure)
                elif measure=='T':
                        return self._eval(db_segmentation,db_eval_t_stab,measure)
                else:
                        raise Exception("Unknown measure=[%s]. Valid options are measure={J,F,T}"%measure)
'''

Overwriting /content/davis/python/lib/davis/dataset/loader.py


In [61]:
%%writefile /content/davis/python/lib/davis/config.py
# ----------------------------------------------------------------------------
# A Benchmark Dataset and Evaluation Methodology for Video Object Segmentation
#-----------------------------------------------------------------------------
# Copyright (c) 2016 Federico Perazzi
# Licensed under the BSD License [see LICENSE for details]
# Written by Federico Perazzi
# Adapted from FAST-RCNN (Ross Girshick)
# ----------------------------------------------------------------------------

""" Configuration file."""

import os
import os.path as osp

import sys
import yaml
from easydict import EasyDict as edict


__C = edict()

# Public access to configuration settings
cfg = __C

# Paths to dataset folders
__C.PATH = edict()

# Dataset Resolution  Available: 1080p,480p
__C.RESOLUTION="480p"

# Root folder of project
__C.PATH.ROOT_DIR = osp.abspath(osp.join(osp.dirname(__file__), '..', '..','..'))

# Data folder
__C.PATH.DATA_DIR       = osp.abspath(osp.join(__C.PATH.ROOT_DIR, '../DAVIS'))

__C.PATH.RESULTS_DIR  = osp.abspath(osp.join(__C.PATH.DATA_DIR,  'Results'))

# Resulting segmentation mask folder
__C.PATH.SEGMENTATION_DIR  = osp.abspath(osp.join(__C.PATH.RESULTS_DIR,  'Segmentations', __C.RESOLUTION))

# Evaluation Folder"
__C.PATH.EVAL_DIR     = osp.abspath(osp.join(__C.PATH.RESULTS_DIR,  'Evaluation', __C.RESOLUTION))

# Path to input images
__C.PATH.SEQUENCES_DIR   = osp.join(__C.PATH.DATA_DIR,"JPEGImages",__C.RESOLUTION)

# Path to annotations
__C.PATH.ANNOTATION_DIR  = osp.join(__C.PATH.DATA_DIR,"Annotations",__C.RESOLUTION)

# Paths to files
__C.FILES = edict()

# Path to property file, holding information on evaluation sequences.
__C.FILES.DB_INFO = osp.abspath(osp.join(__C.PATH.DATA_DIR,"Annotations/db_info.yml"))

# Define the set of techniques to be loaded
__C.EVAL_SET="all" # Accepted options [paper,all]

assert __C.EVAL_SET == 'paper' or __C.EVAL_SET == 'all'

# Path to technique file, holding information about benchmark data
__C.FILES.DB_BENCHMARK          = osp.abspath(
                osp.join(__C.PATH.RESULTS_DIR,"Evaluation/db_benchmark.yml"))

__C.N_JOBS = 32

# append path for cpp libraries
def _set_path_to_cpp_libs():
        sys.path.append(osp.abspath(
                osp.join(cfg.PATH.ROOT_DIR,'build/release')))


Overwriting /content/davis/python/lib/davis/config.py


In [56]:
%%writefile /content/davis/python/lib/davis/dataset/utils.py
# ----------------------------------------------------------------------------
# A Benchmark Dataset and Evaluation Methodology for Video Object Segmentation
#-----------------------------------------------------------------------------
# Copyright (c) 2016 Federico Perazzi
# Licensed under the BSD License [see LICENSE for details]
# Written by Federico Perazzi
# ----------------------------------------------------------------------------

import glob
import h5py
import yaml
import warnings

import numpy   as np
import os.path as osp

from easydict        import EasyDict as edict
from davis.parallel  import Parallel,delayed
from collections     import defaultdict,OrderedDict

from davis  import cfg,log,Timer
from davis.dataset.loader import DAVISAnnotationLoader,DAVISSegmentationLoader

def db_statistics(per_frame_values):

        """ Compute mean,recall and decay from per-frame evaluation.

        Arguments:
                per_frame_values (ndarray): per-frame evaluation

        Returns:
                M,O,D (float,float,float):
                        return evaluation statistics: mean,recall,decay.
        """

        # strip off nan values
        with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=RuntimeWarning)
                M = np.nanmean(per_frame_values)
                O = np.nanmean(per_frame_values[1:-1]>0.5)

        # Compute decay as implemented in Matlab
        per_frame_values = per_frame_values[1:-1] # Remove first frame

        N_bins = 4
        ids = np.round(np.linspace(1,len(per_frame_values),N_bins+1)+1e-10)-1;
        ids = ids.astype(np.uint8)

        D_bins = [per_frame_values[ids[i]:ids[i+1]+1] for i in range(0,4)]

        with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=RuntimeWarning)
                D = np.nanmean(D_bins[0])-np.nanmean(D_bins[3])

        return M,O,D

#####################################################
# PERFORM BENCHMARK EVALUATION
#####################################################
def db_eval_sequence(technique,sequence,inputdir,metrics=None):

        """ Perform per-frame sequence evaluation.

        Arguments:
                technique (string): name of the method to be evaluated.
                sequence  (string): name of the sequence to be evaluated.
                inputdir (string): path to the technique folder.

        Returns:
                J,j_M,j_O,j_D: region similarity  (per-frame, mean, recall, decay)
                F,F_M,f_O,f_D: boundary accuracy  (per-frame, mean, recall, decay)
                T,t_M        : temporal stability (per-frame, mean)

        """

        db_sequence     = DAVISAnnotationLoader(cfg,sequence)
        db_segmentation = DAVISSegmentationLoader(cfg,sequence,
                        osp.join(inputdir,technique))

        if metrics is None or 'J' in metrics:
                J,j_M,j_O,j_D = db_sequence.eval(db_segmentation,'J')
        else:
                J,j_M,j_O,j_D = [np.nan],np.nan,np.nan,np.nan

        if metrics is None or 'F' in metrics:
                F,f_M,f_O,f_D = db_sequence.eval(db_segmentation,'F')
        else:
                F,f_M,f_O,f_D = [np.nan],np.nan,np.nan,np.nan

        db_sequences_t_eval = map(lambda s: s.name if s.eval_t else None,
                        db_read_sequences())

        if sequence in db_sequences_t_eval and (metrics is None or 'T' in metrics):
                T,t_M,_,_ = db_sequence.eval(db_segmentation,'T')
        else:
                T,t_M = np.ones_like(J[1:])*np.nan, np.nan


        return  J,j_M,j_O,j_D,F,f_M,f_O,f_D,T,t_M

def db_eval(techniques,sequences,inputdir=cfg.PATH.SEGMENTATION_DIR,metrics=None):

        """ Perform per-frame sequence evaluation.

        Arguments:
                techniques (string,list): name(s) of the method to be evaluated.
                sequences  (string,list): name(s) of the sequence to be evaluated.
                inputdir  (string): path to the technique(s) folder.

        Returns:
                db_eval_dict[method][measure][sequence] (dict): evaluation results.

        """

        if isinstance(techniques,str): techniques = [techniques]
        if isinstance(sequences,str):  sequences  = [sequences]

        ndict        = lambda: defaultdict(ndict)
        db_eval_dict = ndict()

        # RAW, per-frame evaluation
        timer = Timer()
        log.info("Number of cores allocated: %d"%cfg.N_JOBS)
        for technique in techniques:
                log.info('Evaluating technique: "%s"'%technique)
                timer.tic()

                J,j_M,j_O,j_D,F,f_M,f_O,f_D,T,t_M = \
                                 zip(*Parallel(n_jobs=cfg.N_JOBS)(delayed(db_eval_sequence)(
                        technique,sequence,inputdir,metrics) for sequence in sequences))
                log.info('Processing time: "%.3f"'%timer.toc())

                # STORE RAW EVALUATION
                for seq_id,sequence in enumerate(sequences):
                        db_eval_dict[technique]['J'][sequence] = J[seq_id]
                        db_eval_dict[technique]['F'][sequence] = F[seq_id]
                        db_eval_dict[technique]['T'][sequence] = T[seq_id]

        return db_eval_dict

#####################################################
# READ DATA
#####################################################
def db_read_info():
        """ Read dataset properties from file."""
        with open(cfg.FILES.DB_INFO,'r') as f:
                return edict(yaml.load(f))

def db_read_benchmark(db_name=cfg.FILES.DB_BENCHMARK):
        """ Read benchmark data from file."""

        with open(cfg.FILES.DB_BENCHMARK,'r') as f:
                db_benchmark = edict(yaml.load(f.read()))
                if cfg.EVAL_SET == 'paper':
                        db_benchmark['techniques'] = filter(
                                        lambda technique: technique.eval_set=='paper',db_benchmark['techniques'])
                return db_benchmark

def db_read_sequences():
        """ Read list of sequences. """
        return db_read_info().sequences

def db_read_attributes():
        """ Read list of sequences. """
        return db_read_info().attributes

def db_read_techniques(db_name=cfg.FILES.DB_BENCHMARK):
        """ Read list of benchmarked techniques."""
        return db_read_benchmark().techniques

def db_read_eval(technique=None,measure=None,
                sequence=None,raw_eval=False,inputdir=cfg.PATH.EVAL_DIR):

        """ Read per-frame evaluation from file.

        Arguments:

                technique (string,list): name(s) of the method to be evaluated.
                measure   (string,list): name(s) of measure to be computed (M,O,D).
                sequence  (string,list): name(s) of the sequence to be evaluated.
                inputdir  (string)     : path to the technique(s) folder.
                raw_eval  (bool)       : when False return compute measure statistics (mean,recall,decay).

        Returns:
                db_eval_dict[method][measure][sequence] (dict): evaluation results.

        """

        def _listify(arg,func):
                if isinstance(arg,str):
                        return [arg]
                elif arg is None:
                        return func()
                else:
                        return arg

        measures   = _listify(measure,
                        lambda: db_read_benchmark().measures)

        techniques = _listify(technique,
                        lambda: [t.name for t in db_read_techniques()])

        sequences = _listify(sequence,
                        lambda: [s.name for s in db_read_sequences()])

        # Compute statistics if not raw_eval else eval_func is is identity
        eval_func = db_statistics \
                        if not raw_eval else lambda i:i

        db = {}

        for t in techniques:
                db[t] = OrderedDict()
                db_h5 = h5py.File(osp.join(inputdir,t+'.h5'),'r')

                for m in measures:
                        db[t][m] = OrderedDict()
                        for s in sequences:
                                db[t][m][s] = eval_func(db_h5[m][s][...])

        return db

#######################################################
# SAVE DATA
#######################################################
def db_save_eval(db_eval_dict,outputdir=cfg.PATH.EVAL_DIR):

        """ Save per-frame evaluation to HDF5 file.

        Arguments:
                db_eval_dict[method][measure][sequence] (dict): evaluation results.
                outputdir: destination folder of the output files (one for each technique).

        """


        for technique in db_eval_dict.keys():
                outfilename = osp.join(outputdir,technique + ".h5")
                log.info("Saving evaluation in: %s"%outfilename)
                db_hdf5 = h5py.File(outfilename,'w')
                for measure in db_eval_dict[technique].keys():
                        for sequence,val in db_eval_dict[technique][measure].iteritems():
                                db_hdf5["%s/%s"%(measure,sequence)] = val

                db_hdf5.close()

# HELPER FUNCTION NOT MEANT TO BE USED
def db_save_techniques(db_eval_dict,filename=cfg.FILES.DB_BENCHMARK):

        db_techniques = {'techniques':[],
                                                 'types'  :['preprocessing','semisupervised','supervised'],
                                                 'measures':['J','F','T']}

        method_type = [('preprocessing' , ['mcg', 'sf-lab','sf-mot']),
                                                                 ('unsupervised'  , ['nlc', 'cvos', 'trc', 'msg', 'key', 'sal', 'fst']),
                                                                 ('semisupervised', ['tsp', 'sea', 'hvs', 'jmp', 'fcp','bvs'])]

        for mtype,techniques in method_type:
                for technique in techniques:
                        scores = db_eval_dict[technique]


                        def gmr(arg):
                                return np.round(np.nanmean(arg.values(),axis=0),3).tolist()

                        db_techniques['techniques'].append(
                                        {
                                                'name'       : technique,
                                                'type'       : mtype,
                                                'J'          : gmr(scores['J']),
                                                'F'          : gmr(scores['F']),
                                                'T'          : [gmr(scores['T'])[0],np.nan,np.nan]
                                        })

        with open(filename,'w') as f:
                f.write(yaml.dump(db_techniques))

def db_eval_view(db_eval_dict,technique,
                summary=False,eval_set='all'):

        db_sequences = db_read_sequences()

        from prettytable import PrettyTable as ptable
        table = ptable(["Sequence"] + ['J(M)','J(O)','J(D)','F(M)','F(O)','F(D)','T(M)'])

        X = []
        for key,values in db_eval_dict[technique].iteritems():
                X.append(db_eval_dict[technique][key].values())

        X = np.hstack(X)[:,:7]
        if not summary:
                for s,row in zip(db_sequences,X):
                        if eval_set == 'all' or s.set == eval_set:
                                table.add_row([s.name]+ ["{: .3f}".format(n) for n in row])


        set_ids = [seq_id for seq_id,seq in enumerate(db_sequences)
                        if eval_set == 'all' or seq.set == eval_set]

        print(set_ids)

        table.add_row(['Average'] + ["{: .3f}".format(n)
                for n in np.nanmean(X[set_ids],axis=0)])

        print("\n" + str(table) + "\n")
        return str(table)

Overwriting /content/davis/python/lib/davis/dataset/utils.py


In [30]:
%%writefile /content/davis/python/lib/davis/measures/__init__.py
from davis.measures.jaccard     import db_eval_iou
from davis.measures.t_stability import db_eval_t_stab
from davis.measures.f_boundary  import db_eval_boundary

Overwriting /content/davis/python/lib/davis/measures/__init__.py


In [19]:
%%writefile /content/davis/python/lib/davis/__init__.py
#-----------------------------------------------------------------------------
# Copyright (c) 2016 Federico Perazzi
# Licensed under the BSD License [see LICENSE for details]
# Written by Federico Perazzi
# ----------------------------------------------------------------------------
from davis.logger import logging as log
from davis.timer import Timer
from davis.config import cfg,_set_path_to_cpp_libs
_set_path_to_cpp_libs()

from davis.dataset.loader import DAVISAnnotationLoader,DAVISSegmentationLoader

Overwriting /content/davis/python/lib/davis/__init__.py


In [44]:
%%writefile /content/davis/python/lib/davis/dataset/__init__.py
# ----------------------------------------------------------------------------
# A Benchmark Dataset and Evaluation Methodology for Video Object Segmentation
#-----------------------------------------------------------------------------
# Copyright (c) 2016 Federico Perazzi
# Licensed under the BSD License [see LICENSE for details]
# Written by Federico Perazzi
# ----------------------------------------------------------------------------

from davis.dataset.loader import DAVISAnnotationLoader,DAVISSegmentationLoader

#from davis.dataset.utils import db_statistics,db_eval,db_read_info,db_read_benchmark,db_read_eval,\
#               db_read_techniques,db_read_eval,db_save_eval,db_read_sequences,db_save_techniques,db_eval_view

Overwriting /content/davis/python/lib/davis/dataset/__init__.py


In [39]:
%%writefile /content/davis/python/lib/davis/measures/t_stability.py
import sys
import numpy as np
import scipy.spatial.distance as ssd
from tstab import *

def get_bijective_pairs(pairs,costmat):
        bij_pairs = bij_pairs_one_dim(pairs, costmat,0)
        bij_pairs = bij_pairs_one_dim(bij_pairs, costmat.T,1)
        return bij_pairs

def bij_pairs_one_dim(pairs, costmat, left_or_right):

        bij_pairs = []
        ids1      = np.unique(pairs[:,left_or_right])

        for ii in range(len(ids1)):
                curr_pairs = pairs[pairs[:,left_or_right]==ids1[ii],:].astype(np.int)
                curr_costs = costmat[curr_pairs[:,left_or_right], curr_pairs[:,1-left_or_right]]
                b = np.argmin(curr_costs)
                bij_pairs.append(curr_pairs[b])

        return np.array(bij_pairs)

def hist_cost_2(BH1,BH2):

        nsamp1,nbins=BH1.shape
        nsamp2,nbins=BH2.shape

        eps  = 2.2204e-16
        BH1n = BH1 / (np.sum(BH1,axis=1,keepdims=True)+eps)
        BH2n = BH2 / (np.sum(BH2,axis=1,keepdims=True)+eps)

        tmp1 = np.tile(np.transpose(np.atleast_3d(BH1n),[0,2,1]),(1,nsamp2,1))
        tmp2 = np.tile(np.transpose(np.atleast_3d(BH2n.T),[2,1,0]),(nsamp1,1,1))
        HC = 0.5*np.sum((tmp1-tmp2)**2/(tmp1+tmp2+eps),axis=2)

        return HC

def sc_compute(Bsamp,Tsamp,mean_dist,nbins_theta,nbins_r,r_inner,r_outer,out_vec):
        in_vec = (out_vec==0).ravel()
        nsamp = Bsamp.shape[1]
        r_array=ssd.squareform(ssd.pdist(Bsamp.T)).T

        theta_array_abs0=Bsamp[1,:].reshape(-1,1).dot(np.ones((1,nsamp))) - \
                        np.ones((nsamp,1)).dot(Bsamp[1,:].reshape(1,-1))

        theta_array_abs1=Bsamp[0,:].reshape(-1,1).dot(np.ones((1,nsamp))) - \
                        np.ones((nsamp,1)).dot(Bsamp[0,:].reshape(1,-1))

        theta_array_abs = np.arctan2(theta_array_abs0,theta_array_abs1).T
        theta_array=theta_array_abs-Tsamp.T.dot(np.ones((1,nsamp)))

        if mean_dist is None:
                mean_dist = np.mean(r_array[in_vec].T[in_vec].T)

        r_array_n = r_array / mean_dist

        r_bin_edges=np.logspace(np.log10(r_inner),np.log10(r_outer),nbins_r)
        r_array_q=np.zeros((nsamp,nsamp))

        for m in range(int(nbins_r)):
                r_array_q=r_array_q+(r_array_n<r_bin_edges[m])

        fz = r_array_q > 0
        theta_array_2 = np.fmod(np.fmod(theta_array,2*np.pi)+2*np.pi,2*np.pi)
        theta_array_q = 1+np.floor(theta_array_2/(2*np.pi/nbins_theta))

        nbins=nbins_theta*nbins_r
        BH=np.zeros((nsamp,nbins))
        count = 0
        for n in range(nsamp):
                fzn=fz[n]&in_vec
                Sn = np.zeros((nbins_theta,nbins_r))
                coords = np.hstack((theta_array_q[n,fzn].reshape(-1,1),
                        r_array_q[n,fzn].astype(np.int).reshape(-1,1)))

                # SLOW...
                #for i,j in coords:
                        #Sn[i-1,j-1] += 1

                # FASTER
                ids = np.ravel_multi_index((coords.T-1).astype(np.int),Sn.shape)
                Sn  = np.bincount(ids.ravel(),minlength = np.prod(Sn.shape)).reshape(Sn.shape)


                BH[n,:] = Sn.T[:].ravel()

        return BH.astype(np.int),mean_dist

def db_eval_t_stab(fgmask,ground_truth,timing=True):
        """

        Calculates the temporal stability index between two masks

        Arguments:
                                        fgmask (ndarray):  Foreground Object mask at frame t
                ground_truth (ndarray):  Foreground Object mask at frame t+1

        Return:
                                                         T (ndarray):  Temporal (in-)stability
           raw_results (ndarray):  Supplemental values
        """

        cont_th = 3
        cont_th_up = 3

        # Shape context parameters
        r_inner     = 1.0/8.0
        r_outer     = 2.0
        nbins_r     = 5.0
        nbins_theta = 12.0

        poly1 = mask2poly(fgmask,cont_th)
        poly2 = mask2poly(ground_truth,cont_th)

        if len(poly1.contour_coords) == 0 or \
                        len(poly2.contour_coords) == 0:
                return np.nan

        Cs1 = get_longest_cont(poly1.contour_coords)
        Cs2 = get_longest_cont(poly2.contour_coords)

        upCs1 = contour_upsample(Cs1,cont_th_up)
        upCs2 = contour_upsample(Cs2,cont_th_up)

        scs1,_=sc_compute(upCs1.T,np.zeros((1,upCs1.shape[0])),None,
                        nbins_theta,nbins_r,r_inner,r_outer,np.zeros((1,upCs1.shape[0])))

        scs2,_=sc_compute(upCs2.T,np.zeros((1,upCs2.shape[0])),None,
                        nbins_theta,nbins_r,r_inner,r_outer,np.zeros((1,upCs2.shape[0])))

        # Match with the 0-0 alignment
        costmat              = hist_cost_2(scs1,scs2)
        pairs ,max_sx,max_sy = match_dijkstra(np.ascontiguousarray(costmat))


        # Shift costmat
        costmat2 = np.roll(costmat ,-(max_sy+1),axis=1)
        costmat2 = np.roll(costmat2,-(max_sx+1),axis=0)

        # Redo again with the correct alignment
        pairs,_,_ = match_dijkstra(costmat2)

        # Put the pairs back to the original place
        pairs[:,0] = np.mod(pairs[:,0]+max_sx+1, costmat.shape[0])
        pairs[:,1] = np.mod(pairs[:,1]+max_sy+1, costmat.shape[1])

        pairs = get_bijective_pairs(pairs,costmat)

        pairs_cost = costmat[pairs[:,0], pairs[:,1]]
        min_cost   = np.average(pairs_cost)

        return min_cost


Overwriting /content/davis/python/lib/davis/measures/t_stability.py
